In [5]:
import pandas as pd
import numpy as np

In [19]:
cols = ['bayesian_prediction_std_bal_acc',
        'bayesian_prediction_std_f1_score', 'bayesian_prediction_std_auc',
        'bayesian_entropy_bal_acc', 'bayesian_entropy_f1_score',
        'bayesian_entropy_auc']
df_with = pd.read_csv('bayesian_uncertainty_with_unknown.csv')
#df_with_results = df_with.groupby(['inside_labels', 'unknown_labels']).agg([np.mean, np.std])[cols]
df_with_results = df_with.groupby(['inside_labels', 'unknown_labels']).mean()[cols]

In [20]:
df_without = pd.read_csv('bayesian_uncertainty_without_unknown.csv')
#df_without_results = df_without.groupby(['inside_labels', 'unknown_labels']).agg([np.mean, np.std])[cols]
df_without_results = df_without.groupby(['inside_labels', 'unknown_labels']).mean()[cols]

In [34]:
cols = ['inside', 'outside',
        'with_std_acc', 'out_std_acc',
        'with_std_auc', 'out_std_auc',
        'with_entropy_acc', 'out_entropy_acc',
        'with_entropy_auc', 'out_entropy_auc']
results = pd.DataFrame(columns=cols)
for (wkey, *wrow), (okey, *orow) in zip(df_with_results.itertuples(), df_without_results.itertuples()):
    assert wkey == okey
    results.loc[len(results)] = [
        wkey[0], wkey[1],
        wrow[0], orow[0],
        wrow[2], orow[2],
        wrow[3], orow[3],
        wrow[5], orow[5]
    ]

In [35]:
results

,inside,outside,with_std_acc,out_std_acc,with_std_auc,out_std_auc,with_entropy_acc,out_entropy_acc,with_entropy_auc,out_entropy_auc
0,"[0, 1, 4, 8]","[7, 9]",0.788317,0.745496,0.863209,0.807197,0.775390,0.740770,0.845955,0.804086
1,"[0, 1, 6, 7]","[3, 9]",0.916986,0.834277,0.965058,0.898990,0.914993,0.829429,0.968852,0.895113
2,"[0, 2, 4, 6]","[5, 9]",0.911990,0.837949,0.957000,0.900818,0.915502,0.837641,0.971875,0.900083
3,"[0, 3, 6, 9]","[1, 5]",0.755404,0.713647,0.827527,0.774093,0.746369,0.712626,0.817967,0.773164
4,"[0, 3, 7, 8]","[5, 6]",0.857855,0.849689,0.922862,0.916942,0.847801,0.849358,0.916855,0.917121
5,"[0, 4, 5, 6]","[1, 3]",0.857657,0.811952,0.928185,0.890709,0.855409,0.809892,0.935403,0.888444
6,"[0, 5, 6, 9]","[2, 3]",0.790826,0.716698,0.858562,0.771928,0.789691,0.715681,0.868484,0.772306
7,"[1, 2, 6, 8]","[4, 9]",0.848110,0.761987,0.915455,0.820964,0.846873,0.760882,0.916858,0.820752
8,"[1, 3, 5, 7]","[2, 8]",0.865305,0.705070,0.927163,0.741156,0.866216,0.699320,0.933106,0.732646
9,"[1, 5, 6, 7]","[4, 8]",0.833296,0.743053,0.902974,0.807804,0.822648,0.736807,0.892972,0.800346


In [36]:
results.to_csv('final_results.csv', index=False)